<a href="https://colab.research.google.com/github/FarshadBorjalizade/Face-recognitionvia-Deep-Stacked-Denoising-Autoencoders/blob/main/Cell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unrar x "/content/drive/MyDrive/Dataset/image_processing_HW3.rar" "/content/sample_data/Img_HW3"

In [ ]:
import tensorflow as tf
import PIL
import pandas as pd
import numpy as np
import os
import cv2
import random
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow import keras
from matplotlib import pyplot as plt
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from skimage import io
%matplotlib inline

In [ ]:
dataset_dir = r'/content/sample_data/Img_HW3'

**Show some randome image before Augmentation**



In [ ]:
plt.figure(figsize=(20,20))

img_folder = os.path.join(dataset_dir, 'Balle_cell_1')
#print(img_folder)
for i in range(5):
    file = random.choice(os.listdir(img_folder))
    image_path= os.path.join(img_folder, file)
    img=io.imread(image_path)
    ax=plt.subplot(1,5,i+1)
    ax.title.set_text(file)
    plt.imshow(img)

In [ ]:
print(type(img), img.dtype)
print(img.shape)

In [ ]:
folder_list = os.listdir(dataset_dir)
folder_list

In [ ]:
data = []
labels = []
class_labels = {'Not_balle_cell_0': 0, 'Balle_cell_1': 1}

**Image agmentation**

In [ ]:
datagen1 = ImageDataGenerator(        
            rotation_range=45,
            width_shift_range=0.2,  
            height_shift_range=0.2,    
            shear_range=0.2,        
            zoom_range=0.2,        
            horizontal_flip=True,         
            fill_mode='constant', cval=125)

In [ ]:
BATCH_SIZE = 32
TARGET_SIZE = (257, 257, 3)
datagen1 = datagen1.flow_from_directory(
    dataset_dir,
    target_size=TARGET_SIZE[:2],
    batch_size=BATCH_SIZE,
    class_mode='categorical')

In [ ]:
datagen2 = ImageDataGenerator(        
            rotation_range=45,
            width_shift_range=0.2,  
            height_shift_range=0.2,    
            shear_range=0.2,        
            zoom_range=0.2,        
            horizontal_flip=True,         
            fill_mode='constant', cval=125)

In [ ]:
for class_folder in folder_list:
    image_list = os.listdir(os.path.join(dataset_dir, class_folder))
    
    for image_name in image_list:
        current_image_path = os.path.join(dataset_dir, class_folder, image_name)
        image = io.imread(current_image_path)
        #print(image.shape)
        image = image.reshape((1, ) + image.shape)
        #print(image.shape)
        j = 0
        for batch in datagen2.flow(image, batch_size=16,
                                  save_to_dir=os.path.join(dataset_dir, class_folder),
                                  save_prefix='aug',
                                  save_format='tif'):    
            j += 1    
            if j > 3:        
                break

In [ ]:
plt.figure(figsize=(20,20))

img_folder = os.path.join(dataset_dir, 'Balle_cell_1')
#print(img_folder)
for i in range(5):
    file = random.choice(os.listdir(img_folder))
    image_path= os.path.join(img_folder, file)
    img=io.imread(image_path)
    ax=plt.subplot(1,5,i+1)
    ax.title.set_text(file)
    plt.imshow(img)

In [ ]:
plt.figure(figsize=(20,20))

img_folder = os.path.join(dataset_dir, 'Not_balle_cell_0')
#print(img_folder)
for i in range(5):
    file = random.choice(os.listdir(img_folder))
    image_path= os.path.join(img_folder, file)
    img=io.imread(image_path)
    ax=plt.subplot(1,5,i+1)
    ax.title.set_text(file)
    plt.imshow(img)

In [ ]:
datagen3 = ImageDataGenerator(        
            rotation_range=45,
            width_shift_range=0.2,  
            height_shift_range=0.2,    
            shear_range=0.2,        
            zoom_range=0.2,        
            horizontal_flip=True,         
            fill_mode='constant', cval=125)

In [ ]:
BATCH_SIZE = 32
TARGET_SIZE = (257, 257, 3)
datagen3 = datagen3.flow_from_directory(
    '/content/sample_data/Img_HW3',
    target_size=TARGET_SIZE[:2],
    batch_size=BATCH_SIZE,
    class_mode='categorical')

In [ ]:
my_data = []
my_labels = []
for class_folder in folder_list:
    image_list = os.listdir(os.path.join(dataset_dir, class_folder))
    #print(image_list)
    for image_name in image_list:
        current_image_path = os.path.join(dataset_dir, class_folder, image_name)
        image = cv2.imread(current_image_path)
        image = cv2.resize(image, (257, 257))
        
        my_data.append(image)
        my_labels.append(class_labels[class_folder])

In [ ]:
len(my_data), type(my_data), my_data[0].dtype

In [ ]:
len(my_labels), type(my_labels)

In [ ]:
my_data = np.array(my_data)
my_labels = np.array(my_labels)

In [ ]:
my_data.shape, type(my_data), my_data.dtype

In [ ]:
my_labels.shape, type(my_labels), my_labels.dtype

In [ ]:
my_labels[0], plt.imshow(cv2.cvtColor(my_data[0], cv2.COLOR_BGR2RGB))

In [ ]:
my_labels[800], plt.imshow(cv2.cvtColor(my_data[800], cv2.COLOR_BGR2RGB))

In [ ]:
prev_state = np.random.get_state()
np.random.shuffle(my_data)
np.random.set_state(prev_state)
np.random.shuffle(my_labels)

In [ ]:
my_labels[0], plt.imshow(cv2.cvtColor(my_data[0], cv2.COLOR_BGR2RGB))

In [ ]:
my_labels[800], plt.imshow(cv2.cvtColor(my_data[800], cv2.COLOR_BGR2RGB))

In [ ]:
train_x, train_y = my_data[:1008], my_labels[:1008]

In [ ]:
val_x, val_y = my_data[1008:], my_labels[1008:]

In [ ]:
type(train_x), train_x.dtype, train_x.shape

In [ ]:
type(train_y), train_y.dtype, train_y.shape

In [ ]:
type(val_x), val_x.dtype, val_x.shape

In [ ]:
type(val_y), val_y.dtype, val_y.shape

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
train_y = to_categorical(train_y, num_classes=2)
train_y.shape, train_y.dtype

In [ ]:
train_y[0]

In [ ]:
val_y = to_categorical(val_y, num_classes=2)
val_y.shape, val_y.dtype

In [ ]:
train_x = train_x.astype('float32') / 255.0
train_x.shape, train_x.dtype

In [ ]:
val_x = val_x.astype('float32') / 255.0
val_x.shape, val_x.dtype

In [ ]:
train_x.mean(), train_x.max(), train_x.min()

In [ ]:
val_x.mean(), val_x.max(), val_x.min()

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, Flatten, GlobalAveragePooling2D, Dropout, ReLU, ZeroPadding2D

In [ ]:
input_layer = Input(shape=(257, 257, 3))

x = Conv2D(filters=256, kernel_size=(3, 3), strides=2, padding='same')(input_layer)
x = ReLU()(x)
x = ZeroPadding2D(padding=(0, 0))(x)
x = MaxPool2D(pool_size=(2, 2), strides=2 , padding='valid')(x)

x = ZeroPadding2D(padding=(2, 2))(x)
x = Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding='valid')(x)
x = ReLU()(x)
x = MaxPool2D(pool_size=(2, 2), strides=2 , padding='valid')(x)

x = ZeroPadding2D(padding=(3, 3))(x)
x = Conv2D(filters=96, kernel_size=(5, 5), strides=1, padding='valid')(x)
x = ReLU()(x)

x = ZeroPadding2D(padding=(2, 2))(x)
x = Conv2D(filters=96, kernel_size=(5, 5), strides=1, padding='valid')(x)
x = ReLU()(x)

x = ZeroPadding2D(padding=(2, 2))(x)
x = Conv2D(filters=32, kernel_size=(3, 3), strides=1, padding='valid')(x)
x = ReLU()(x)
x = MaxPool2D(pool_size=(3, 3), strides=2 , padding='valid')(x)

x = GlobalAveragePooling2D()(x)

x = Dense(units=1000, activation='relu')(x)
x = ReLU()(x)
x = Dropout(0.5)(x)

predictions_layer = Dense(units=2, activation='softmax')(x)

my_cell_classifier_model = Model(inputs=input_layer, outputs=predictions_layer)
my_cell_classifier_model.summary()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
my_cell_classifier_model.compile(optimizer=opt , loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_accuracy', mode='max', verbose=0, patience=151, restore_best_weights=True)

In [ ]:
my_cell_classifier_model.fit(x=train_x, y=train_y,
                             epochs=107,
                             batch_size=100,
                             shuffle=True,
                             validation_data=(val_x, val_y))

In [ ]:
from tensorflow.keras.models import load_model, save_model

In [ ]:
save_model(my_cell_classifier_model, '/content/drive/MyDrive', 'balle_cell.h5' )

In [ ]:
model_loss_history = pd.DataFrame({'loss': my_cell_classifier_model.history.history['loss'],
                                   'val_loss': my_cell_classifier_model.history.history['val_loss']})

In [ ]:
model_loss_history.plot()

In [ ]:
model_accuracy_history = pd.DataFrame({'accuracy': my_cell_classifier_model.history.history['accuracy'],
                                       'val_accuracy': my_cell_classifier_model.history.history['val_accuracy']})

In [ ]:
model_accuracy_history.plot()

In [ ]:
model_accuracy_history['val_accuracy'].max()

In [ ]:
my_cell_classifier_model.evaluate(x=val_x, y=val_y)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
model_predictions = my_cell_classifier_model.predict(val_x)

In [ ]:
model_predictions

In [ ]:
model_predictions.shape

In [ ]:
model_sparse_predictions = np.zeros(shape=model_predictions.shape)
predictions_sparse_labels= np.argmax(model_predictions, axis=1)

In [ ]:
predictions_sparse_labels

In [ ]:
val_y =  np.argmax(val_y, axis=1)
val_y

In [ ]:
print(classification_report(y_true=val_y, y_pred=predictions_sparse_labels))

In [ ]:
cf_matrix = confusion_matrix(y_true=val_y, y_pred=predictions_sparse_labels)


In [ ]:
import seaborn as sns
labels = ['True Neg','False Pos','False Neg','True Pos']
#categories = ['Normal', 'Abnormal']
group_counts = ['{0:0.0f}'.format(value) for value in cf_matrix.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in zip(labels, group_counts, group_percentages)]
#print(labels)
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')